<a href="https://colab.research.google.com/github/qu1r0ra/philippine-machine-translation/blob/chore%2Ftrain-transformer/notebooks/02b_modeling_nmt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modeling in Neural Machine Translation (NMT)

## Notes

The author who trained the Transformer model did not have sufficient computational resources to train locally, so he decided to train it via Google Colab.

Unfortunately, the free plan in Colab also did not suffice for training as the free GPUs have rate limits, so the author also decided to purchase compute units to access said GPUs. You may also need Colab compute units to replicate the results of this notebook.

## Environment Setup

First, let us uninstall Colab's preinstalled `torch`, `torchvision`, and `torchaudio` packages.

We need to downgrade them as the `OpenNMT-py` version that the authors used for training requires older versions.

**NOTE:** You only need to run this the first time.

In [ ]:
!pip uninstall torch torchvision torchaudio -y

Found existing installation: torch 2.8.0+cu126
Uninstalling torch-2.8.0+cu126:
  Successfully uninstalled torch-2.8.0+cu126
Found existing installation: torchvision 0.23.0+cu126
Uninstalling torchvision-0.23.0+cu126:
  Successfully uninstalled torchvision-0.23.0+cu126
Found existing installation: torchaudio 2.8.0+cu126
Uninstalling torchaudio-2.8.0+cu126:
  Successfully uninstalled torchaudio-2.8.0+cu126


Next, let us install `condacolab` so we can install the older package versions via `conda`.

The authors tried using `pip` to install the older versions but they were somehow unlisted and thus could not be installed.

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


Now we can install the older versions required by our `OpenNMT-py` version.

In [ ]:
!conda install --quiet pytorch==2.1.1 torchvision==0.16.1 torchaudio==2.1.1 \
  pytorch-cuda=11.8 -c pytorch -c nvidia

Channels:
 - pytorch
 - nvidia
 - conda-forge
Platform: linux-64
Solving environment: ...working... done

# All requested packages already installed.



In [ ]:
!pip install OpenNMT-py==3.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.0/37.0 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 132.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 104.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 842.9/842.9 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 107.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 80.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 76.3 M

Lastly, let us create the folders where we will be uploading our data.

In [ ]:
!mkdir -p data outputs

## Data Upload

Next, we need to upload the necessary files to train our model.

At this point, it is assumed that:
1. You have cloned the GitHub repository https://github.com/qu1r0ra/philippine-machine-translation.
2. You have gone through `00_setup.ipynb` and `01b_preprocessing_nmt.ipynb` in the repository.

To prepare the data needed for this notebook, simply follow these instructions:
1. Upload `config.yaml` from the repository to the Colab filesystem (`/content`). You know you are in the right path if you can also see the file `condacolab_install.log`.
3. Upload `train.src`, `train.tgt`, `valid.src`, and `valid.tgt` from `data/processed/` in the repository to the `data/` folder in Colab.

Let us check the first few lines of our training and validation data.

In [ ]:
!head -n 5 data/train.src

ug ang bisan unsa nga walay kapay ug himbis ayaw ninyo kaona hugaw kini alang kaninyo
kay ang akong mga adlaw nangahanaw sama sa aso ug ang akong kabukogan nagdilaab sama sa hudno
busa karon inigabot niining sulat diha kanimo sa nakita nimo nga ang mga anak sa inyong agalon uban kaninyo ug anaa kaninyo ang mga karwahi ug mga kabayo ug ang mga kinutaang siyudad ug mga hinagiban
ang mga sulat gipadala pinaagi sa mga sinugo ngadto sa tanang mga lalawigan sa hari aron sa paglaglag sa pagpatay ug sa pagpuo sa tanang mga judio batanon ug tigulang mga babaye ug mga kabataan nga gagmay sulod sa usa ka adlaw sa ikanapulo ug tulo ka adlaw sa ikanapulo ug duha nga bulan nga mao ang bulan sa adar ug sa pagkuha sa tanan nila nga mga kabtangan
ug ug kini kini mipadayon ngadto sa habagatan sa tungasan sa akrabim subay sa zin ug mitungas dapit sa habagatan sa kadesbarnea agi sa hesron ngadto sa adar ug miliko paingon sa karka


In [ ]:
!head -n 5 data/train.tgt

pero no comeréis lo que no tiene aletas y escama os será inmundo
porque mis días se desvanecen como el humo y mis huesos cual tizón están quemados
inmediatamente que lleguen estas cartas a vosotros como tenéis a los hijos de vuestro señor y también tenéis carros y gente de a caballo la ciudad fortificada y las armas
y se enviaron las cartas por medio de correos a todas las provincias del rey con la orden de destruir matar y aniquilar a todos los judíos jóvenes y ancianos niños y mujeres y de apoderarse de sus bienes en un mismo día en el día trece del mes duodécimo que es el mes de adar
luego salía hacia el sur de la subida de acrabim pasaba hacia zin y subía por el sur hasta cadesbarnea pasando por hezrón subía hacia adar y daba vuelta a carca


In [ ]:
!head -n 5 data/valid.src

unya ang paraon mipatawag kang moises ug miingon panglakaw kamo magalagad aron kamo sa ginoo ang inyong mga kabataan makauban kaninyo apan kinahanglang ibilin ninyo ang inyong mga karnero ug ang inyong mga baka
kini mao ang ihalad ninyo sa ginoo sa inyong gitakda nga mga kasaulogan dugang sa mga saad ug sa inyong mga halad nga kinabubuton alang sa inyong mga nga ug alang sunogon sa mga inyong halad nga pagkaon ug alang sa inyong mga halad nga ilimnon ug alang sa inyong inyong halad mga sa pakigdait
ug ang iyang panon sumala sa pagihap
busa si pablo gipagikan dayon sa mga kaigsoonan sa paglakaw padulong sa dagat apan si silas ug si timoteo nagpabilin didto
si jesus miingon kaniya sa pagkatinuod sultihan ko ikaw niini gayong gabhiona sa dili pa motuktugaok ang manok ilimod nimo ako sa makatulo


In [ ]:
!head -n 5 data/valid.tgt

entonces el faraón hizo llamar a moisés y dijo id servid a jehová que solamente queden vuestras ovejas y vuestras vacas vayan también vuestros niños con vosotros
estas cosas ofreceréis a jehová en vuestras fiestas solemnes además de vuestros votos y ofrendas voluntarias de vuestros holocaustos ofrendas y libaciones y de vuestras ofrendas de paz
su cuerpo de ejército según el censo sesenta y dos mil setecientos
entonces los hermanos hicieron que pablo saliera inmediatamente en dirección al mar pero silas y timoteo se quedaron allí
jesús le dijo de cierto te digo que esta noche antes que el gallo cante me negarás tres veces


(will soon write stuff about the data)

## Training a Small Transformer with OpenNMT-py

(insert additional explanation)

The advantage with using `OpenNMT-py` is that we only need to configure the parameters and hyperparameters of the model with a `config.yaml` file. After doing so, the training can be ran with one line.

Normally, we would need to code the architecture and mechanisms of the model ourselves in deep learning frameworks like `PyTorch` and `TensorFlow`. As an aside, `OpenNMT` has implementations in both of these frameworks, but the authors chose the PyTorch implementation, thus named `OpenNMT-py`.

In [ ]:
!pip install "numpy<2.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 123.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.4
    Uninstalling numpy-2.3.4:
      Successfully uninstalled numpy-2.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
!onmt_build_vocab -config config-gru-aug.yaml

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2025-11-02 13:44:24,315 INFO] Counter vocab from 5000 samples.
[2025-11-02 13:44:24,315 INFO] Build vocab on 5000 transformed examples/corpus.
[2025-11-02 13:44:24,520 INFO] Counters src: 7915
[2025-11-02 13:44:24,520 INFO] Counters tgt: 10699


In [ ]:
!onmt_train -config config-gru-aug.yaml

[2025-11-02 13:44:31,904 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2025-11-02 13:44:31,904 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2025-11-02 13:44:31,904 INFO] Missing transforms field for valid data, set to default: [].
[2025-11-02 13:44:31,904 INFO] Parsed 2 corpora from -data.
[2025-11-02 13:44:31,905 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2025-11-02 13:44:31,933 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', 'sa', 'ug', 'ang', 'nga', 'mga', 'iyang']
[2025-11-02 13:44:31,934 INFO] The decoder start token is: <s>
[2025-11-02 13:44:31,934 INFO] Building model...
[2025-11-02 13:44:32,216 INFO] Switching model to float32 for amp/apex_amp
[2025-11-02 13:44:32,216 INFO] Non quantized layer compute is fp32
[2025-11-02 13:44:32,381 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Eleme

This is literally it. Now we just have to wait until the model finishes training.

Do note that training even a small Transformer like this in general may take several hours to even days, depending on various factors such as:
- the size of your dataset
- the parameters and hyperparameters you set in `config.yaml`
- the power of your GPU
- etc.

For the case of CJ, the author who trained this model via Google Colab, it took him (insert time) to train the model with an **L4** GPU provided by Google Colab.

Mount your Google Drive. You need to save the models that you just created.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p /content/drive/MyDrive/opennmt_models
!cp -r outputs /content/drive/MyDrive/opennmt_models/
!cp -r data /content/drive/MyDrive/opennmt_models/

## Training the Same Exact Model Architecture on Non-Augmented Data (i.e., no Noise Injection)

Since we want to compare

In [ ]:
!onmt_build_vocab -config config-gru-base.yaml

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2025-11-02 16:15:33,975 INFO] Counter vocab from 5000 samples.
[2025-11-02 16:15:33,976 INFO] Build vocab on 5000 transformed examples/corpus.
[2025-11-02 16:15:34,182 INFO] Counters src: 8180
[2025-11-02 16:15:34,183 INFO] Counters tgt: 10994


In [ ]:
!onmt_train -config config-gru-base.yaml

[2025-11-02 16:16:03,728 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2025-11-02 16:16:03,729 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2025-11-02 16:16:03,729 INFO] Missing transforms field for valid data, set to default: [].
[2025-11-02 16:16:03,729 INFO] Parsed 2 corpora from -data.
[2025-11-02 16:16:03,729 INFO] Get special vocabs from Transforms: {'src': [], 'tgt': []}.
[2025-11-02 16:16:03,759 INFO] The first 10 tokens of the vocabs are:['<unk>', '<blank>', '<s>', '</s>', 'sa', 'ug', 'ang', 'nga', 'mga', 'iyang']
[2025-11-02 16:16:03,759 INFO] The decoder start token is: <s>
[2025-11-02 16:16:03,759 INFO] Building model...
[2025-11-02 16:16:04,048 INFO] Switching model to float32 for amp/apex_amp
[2025-11-02 16:16:04,048 INFO] Non quantized layer compute is fp32
[2025-11-02 16:16:04,186 INFO] NMTModel(
  (encoder): RNNEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Eleme

Well, that surprisingly took much quicker to train on than the augmented data.